In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn.metrics import accuracy_score


In [2]:
data=pd.read_csv("loan_train.csv")
test_data = pd.read_csv("loan_test.csv")


In [3]:
data=data.dropna(axis=1)
test_data=data.dropna(axis=1)
nunique = data.nunique()
cols_to_drop = nunique[nunique == 1].index
data = data.drop(cols_to_drop, axis=1)
test_data = test_data.drop(cols_to_drop, axis=1)

data["term"] = data["term"].replace('months', '', regex=True).astype('int64')
data["int_rate"] = data["int_rate"].replace('%', '', regex=True).astype(float)
data.drop(["url"], axis=1, inplace=True)

test_data["term"] = test_data["term"].replace(
    'months', '', regex=True).astype('int64')
test_data["int_rate"] = test_data["int_rate"].replace(
    '%', '', regex=True).astype(float)
test_data.drop(["url"], axis=1, inplace=True)

df = data[data["loan_status"] != "Current"]
df["loan_status"].replace(
    {"Fully Paid": "1", "Charged off": "-1", "Charged Off": "-1"}, inplace=True)
df["loan_status"] = df["loan_status"].astype('int64')

test_data = test_data[test_data["loan_status"] != "Current"]
test_data["loan_status"].replace(
    {"Fully Paid": "1", "Charged off": "-1", "Charged Off": "-1"}, inplace=True)
test_data["loan_status"] = test_data["loan_status"].astype('int64')


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-3-abf150ccbd17>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["loan_status"] = df["loan_status"].astype('int64')


In [4]:
num_cols = data._get_numeric_data()
cols = data.columns
catogory_cols = list(set(cols)-set(num_cols))
len(catogory_cols)
print(catogory_cols)


['zip_code', 'verification_status', 'purpose', 'grade', 'addr_state', 'home_ownership', 'loan_status', 'last_credit_pull_d', 'issue_d', 'earliest_cr_line', 'sub_grade']


In [5]:
# df['Month'] = pd.DatetimeIndex(df['Date']).month
def helper(dataframe):
    new = dataframe["issue_d"].str.split("-", n=1, expand=True)
    dataframe["issue_date"] = new[0]
    dataframe["issue_date"] = dataframe["issue_date"].astype('int64')
    ls = []
    for mon in new[1]:
        ls.append(datetime.datetime.strptime(mon, "%b").month)
    dataframe["issue_month"] = ls
    dataframe["issue_month"] = dataframe["issue_month"].astype('int64')
    dataframe.drop(columns=["issue_d"], inplace=True)

    new = dataframe["last_credit_pull_d"].str.split("-", n=1, expand=True)
    dataframe["last_credit_pull_date"] = new[0]
    dataframe["last_credit_pull_date"] = dataframe["last_credit_pull_date"].astype(
        'int64')
    ls = []
    for mon in new[1]:
        ls.append(datetime.datetime.strptime(mon, "%b").month)
    dataframe["last_credit_pull_month"] = ls
    dataframe["last_credit_pull_month"] = dataframe["last_credit_pull_month"].astype(
        'int64')
    dataframe.drop(columns=["last_credit_pull_d"], inplace=True)


In [6]:
helper(df)
helper(test_data)


<ipython-input-5-2a3b1abad52c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["issue_date"] = new[0]
<ipython-input-5-2a3b1abad52c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["issue_date"] = dataframe["issue_date"].astype('int64')
<ipython-input-5-2a3b1abad52c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

In [7]:

num_cols = df._get_numeric_data()
cols = df.columns
catogory_cols = list(set(cols)-set(num_cols))
len(catogory_cols)
print(catogory_cols)
cols_to_be_encoded = ['sub_grade','verification_status',
                      'purpose', 'home_ownership', 'grade']


['zip_code', 'verification_status', 'purpose', 'grade', 'addr_state', 'home_ownership', 'earliest_cr_line', 'sub_grade']


In [8]:
df = pd.get_dummies(df, columns=cols_to_be_encoded)
test_data = pd.get_dummies(test_data, columns=cols_to_be_encoded)


In [9]:
cols_to_be_catogorised = list(set(catogory_cols)-set(cols_to_be_encoded))
print(cols_to_be_catogorised)
df=df.drop(cols_to_be_catogorised,axis=1)
test_data=test_data.drop(cols_to_be_catogorised,axis=1)


['addr_state', 'earliest_cr_line', 'zip_code']


In [10]:
print(df.shape)
print(test_data.shape)

(24301, 94)
(24301, 94)


In [11]:
y_train=df.loan_status
X_train=df.drop(['loan_status'], axis=1)


In [12]:
y_test = test_data.loan_status
x_test = test_data.drop(['loan_status'], axis=1)


PART A

In [13]:
clf = GradientBoostingClassifier(n_estimators=200)
clf.fit(X_train, y_train)
predictions = clf.predict(x_test)
print("number of decision trees =", 500)
print("accuracy =", accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))
print("\n")


number of decision trees = 500
accuracy = 0.9993415908810337
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00      3474
           1       1.00      1.00      1.00     20827

    accuracy                           1.00     24301
   macro avg       1.00      1.00      1.00     24301
weighted avg       1.00      1.00      1.00     24301





the hyperparameters led to this accuracy are the default parameters of the GradientBoostingClassifier class of sklearn, except for n_estimators.
List of hyperparameters and their values:
earning_rate=0.1 (shrinkage).
n_estimators=500 (number of trees).
max_depth=3.
min_samples_split=2.
min_samples_leaf=1.
subsample=1.0.

PART B

In [14]:
nums=[25,50,100,500,1000]
for n in nums:
    clf = GradientBoostingClassifier(n_estimators=n)
    clf.fit(X_train, y_train)
    predictions = clf.predict(x_test)
    print("number of decision trees =",n)
    print("accuracy =",accuracy_score(y_test, predictions))
    print(classification_report(y_test, predictions))
    print("\n")



number of decision trees = 25
accuracy = 0.9857207522324184
              precision    recall  f1-score   support

          -1       1.00      0.90      0.95      3474
           1       0.98      1.00      0.99     20827

    accuracy                           0.99     24301
   macro avg       0.99      0.95      0.97     24301
weighted avg       0.99      0.99      0.99     24301



number of decision trees = 50
accuracy = 0.9920990905724044
              precision    recall  f1-score   support

          -1       1.00      0.94      0.97      3474
           1       0.99      1.00      1.00     20827

    accuracy                           0.99     24301
   macro avg       1.00      0.97      0.98     24301
weighted avg       0.99      0.99      0.99     24301



number of decision trees = 100
accuracy = 0.9969960083947162
              precision    recall  f1-score   support

          -1       1.00      0.98      0.99      3474
           1       1.00      1.00      1.00     2082

PART C

In [17]:

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
dec_predictions = clf.predict(x_test)
print("accuracy =",accuracy_score(y_test, dec_predictions))
print(classification_report(y_test, dec_predictions))


accuracy = 1.0
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00      3474
           1       1.00      1.00      1.00     20827

    accuracy                           1.00     24301
   macro avg       1.00      1.00      1.00     24301
weighted avg       1.00      1.00      1.00     24301

